# Tensorflow tutorial on MNIST

### Resources



*   Tutorial: https://www.tensorflow.org/versions/r1.1/get_started/mnist/beginners
*   Code: https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/tutorials/mnist/mnist_softmax.py

Posts to check later:
*   Really cool post about **dim reduction**: http://colah.github.io/posts/2014-10-Visualizing-MNIST/
*   Visual interpretation of NN: [http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/](http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/)



In [0]:
from matplotlib import pyplot as plt 
%matplotlib notebook
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from time import sleep as pause

ImportError: No module named tensorflow

In [0]:
train_path = '../input/train.csv'
test_path = '../input/train.csv'
#build a csv line parser (https://www.tensorflow.org/get_started/datasets_quickstart)
batch_size = 100
COLUMNS = pd.read_csv(train_path,nrows=1).columns
FIELD_DEFAULTS = [[0.0]]*COLUMNS.size
def _parse_line(line):
    fields = tf.decode_csv(line, FIELD_DEFAULTS)
    features = dict(zip(COLUMNS, fields))
    label = features.pop('label')
    return features#, label

train_dataset = tf.data.TextLineDataset(train_path).skip(1)
train_dataset = train_dataset.map(_parse_line)#.shuffle(1000000).repeat().batch(batch_size)

NameError: name 'pd' is not defined

In [0]:
def train_input_fn(features):
    #dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = tf.data.Dataset.from_tensor_slices(dict(features))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [0]:
sess.run(train_input_fn(train_dataset))

#### Build dataset structures as in the tutorial from the CSVs

In [0]:
train_raw_data = pd.read_csv('../input/train.csv').sample(100)
test_raw_data = pd.read_csv('../input/train.csv').sample(100)

# create tensors
train_labels, train_images = train_raw_data.pop("label").values, train_raw_data.values
test_labels, test_images = test_raw_data.pop("label").values, test_raw_data.values

# apply one hot encoding to the labels
OHEnconder = OneHotEncoder()
train_labels = OHEnconder.fit_transform(train_labels.reshape(-1,1)).toarray()
test_labels = OHEnconder.fit_transform(test_labels.reshape(-1,1)).toarray()

### Build and run Tensorflow session

In [0]:
def normalize(X):#linear rescaling to the range [-1,1]
    rg = (X.max()-X.min())
    return np.zeros(X.shape) if rg==0 else (X-X.min())/rg


# initialize input placeholders and model variables
x = tf.placeholder(tf.float32, [None,784])
y_ = tf.placeholder(tf.float32, [None,10])

W = tf.Variable(tf.zeros([784, 10])) # no hidden layers: input layer fully connected to the output layer
b = tf.Variable(tf.zeros([10]))    

# specify model output
y = tf.matmul(x, W) + b

# define loss function. Calculated from the cross entropy between the estimated output and the true label
# loss_func = tf.reduce_mean( - tf.reduce_sum(y_ * tf.log(tf.softmax(y)),reduction_indices = [1]))# explicit implementation of cross entropy
loss_func = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_,logits=y)) #nuerically stable implementation from tf
loss=[]

# define what to do in each training iteration
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss_func)

#Start a tensorflow session and initialize variables
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

# Define accuracy
calc_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(y_,1)),tf.float32))

In [0]:
# evaluate model accuracy before training. All weights and biases set to 0 so it should classify everythin as a 0
print("Percentage of 0's in the test set: {:.3f}".format(test_labels.sum(axis=0)[0]/test_labels.sum()))

acc =  sess.run(calc_accuracy, feed_dict={x: test_images,y_: test_labels})
print("Model accuracy: {:.3f}".format(acc))

plt.ion()
fig = plt.figure(1,figsize=(10, 7))
plots = []
for i in range(3):
    for j in range(1,4):
        ind = i*3+j
        plt.subplot(3,3,ind)
        plt.axis('off')
        plt.tight_layout()
        plots.append(plt.imshow(np.zeros([28,28]),cmap='plasma',clim=(-1, 1)))
        plt.title(ind-1)

def refresh_plots(list_of_plots, data):
    for ind, plot in enumerate(plots):
        plot.set_data(normalize(data[:,ind]).reshape(28,28))
    fig.canvas.draw()

In [0]:
N=1
for i in range(10):
    loss.append(sess.run(loss_func, feed_dict={x: test_images,y_: test_labels}))
    sess.run(train_step, feed_dict={x: train_images, y_: train_labels})
    if not i%N: #refresh weights plot every N steps
        refresh_plots(plots, normalize(sess.run(W)))
        pause(1)
plt.figure(3)
loss_function_plt = plt.plot(loss)

In [0]:
#W.initializer.run()